# Set Up

In [ ]:
import gseapy as gp
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import decoupler
import squidpy
import warnings
import re
import os

/home/workspace/environment/pbmc_flow_py_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/workspace/environment/pbmc_flow_py_env/lib/python3.9/site-packages/xarray_schema/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [ ]:
# Create directory for outputs
os.makedirs('spot_plots', exist_ok=True)
print("Created directory: 'spot_plots'")

# Reactome Pathways

In [2]:
vis_data = "/home/workspace/EXP-00971/TIS05393-001-010-EXP-00971-SB005/outs"

adata = sc.read_h5ad('filtered_adata.h5ad')

def gmt_to_decoupler(pth) -> pd.DataFrame:
    """Parse a gmt file to a decoupler pathway dataframe."""
    from itertools import chain, repeat
    from pathlib import Path

    pathways = {}

    with Path(pth).open("r") as f:
        for line in f:
            name, _, *genes = line.strip().split("\t")
            pathways[name] = genes

    return pd.DataFrame.from_records(
        chain.from_iterable(zip(repeat(k), v) for k, v in pathways.items()),
        columns=["geneset", "genesymbol"],
    )

# reactome = gmt_to_decoupler("ReactomePathways.gmt")

# Get reactome pathways
msigdb = decoupler.get_resource("MSigDB")
reactome = msigdb.query("collection == 'reactome_pathways'")

In [3]:
search_mapping = {
    "OXIDATIVE_PHOSPHORYLATION": ["oxidative", "phosphorylation", "respiratory", "electron transport"],
    "GLYCOLYSIS": ["glycolysis", "glucose metabolism"],
    "HYPOXIA": ["hypoxia", "HIF"],
    "IMMUNOGLOBULIN": ["immunoglobulin", "antibody", "Ig synthesis", "class switching"],
    "B_CELL": ["B cell", "BCR", "B-cell"],
    "UPR": ["unfolded protein", "UPR", "endoplasmic reticulum", "ER stress", "IRE1", "PERK", "ATF6"],
    "TNFR2_NFKB": ["TNFR2", "TNF", "NF-kB", "NF-kappaB", "noncanonical", "non-canonical"],
}

print("Searching for matching Reactome pathways:")
print("="*80)

all_pathways = sorted(reactome['geneset'].unique().tolist())

for category, keywords in search_mapping.items():
    print(f"\n{category}:")
    matches = []
    for pathway in all_pathways:
        # Case-insensitive search
        if any(keyword.lower() in pathway.lower() for keyword in keywords):
            matches.append(pathway)
    
    if matches:
        for match in matches:
            print(f"  ✓ {match}")
    else:
        print(f"  ✗ No matches found")

Searching for matching Reactome pathways:

OXIDATIVE_PHOSPHORYLATION:
  ✓ REACTOME_ACTIVATION_OF_PPARGC1A_PGC_1ALPHA_BY_PHOSPHORYLATION
  ✓ REACTOME_AEROBIC_RESPIRATION_AND_RESPIRATORY_ELECTRON_TRANSPORT
  ✓ REACTOME_BETA_CATENIN_PHOSPHORYLATION_CASCADE
  ✓ REACTOME_CAMK_IV_MEDIATED_PHOSPHORYLATION_OF_CREB
  ✓ REACTOME_CDK_MEDIATED_PHOSPHORYLATION_AND_REMOVAL_OF_CDC6
  ✓ REACTOME_CREB1_PHOSPHORYLATION_THROUGH_NMDA_RECEPTOR_MEDIATED_ACTIVATION_OF_RAS_SIGNALING
  ✓ REACTOME_CREB1_PHOSPHORYLATION_THROUGH_THE_ACTIVATION_OF_ADENYLATE_CYCLASE
  ✓ REACTOME_CREB1_PHOSPHORYLATION_THROUGH_THE_ACTIVATION_OF_CAMKII_CAMKK_CAMKIV_CASCASDE
  ✓ REACTOME_CREB_PHOSPHORYLATION
  ✓ REACTOME_FOXO_MEDIATED_TRANSCRIPTION_OF_OXIDATIVE_STRESS_METABOLIC_AND_NEURONAL_GENES
  ✓ REACTOME_JNK_C_JUN_KINASES_PHOSPHORYLATION_AND_ACTIVATION_MEDIATED_BY_ACTIVATED_HUMAN_TAK1
  ✓ REACTOME_OXIDATIVE_STRESS_INDUCED_SENESCENCE
  ✓ REACTOME_PHOSPHORYLATION_AND_NUCLEAR_TRANSLOCATION_OF_BMAL1_ARNTL_AND_CLOCK
  ✓ REACTOME_PHOSPH

In [4]:
# Filter duplicates
reactome = reactome[~reactome.duplicated(("geneset", "genesymbol"))]

# Filtering genesets - FIX: Add observed=True
geneset_size = reactome.groupby("geneset", observed=True).size()
gsea_genesets = geneset_size.index[(geneset_size > 15) & (geneset_size < 500)]

decoupler.run_aucell(
    adata,
    reactome,
    source="geneset",
    target="genesymbol",
    use_raw=False,
)

pathways = [
    "REACTOME_CELLULAR_RESPONSE_TO_HYPOXIA",
    "REACTOME_AEROBIC_RESPIRATION_AND_RESPIRATORY_ELECTRON_TRANSPORT",
    "REACTOME_RESPIRATORY_ELECTRON_TRANSPORT",
    "REACTOME_GLUCOSE_METABOLISM",
    "REACTOME_TNF_SIGNALING"
]

adata.obs[pathways] = adata.obsm["aucell_estimate"][pathways]

In [ ]:
spatial_coords = adata.obsm['spatial']

x_coords = spatial_coords[:, 0]
y_coords = spatial_coords[:, 1]

x_range = x_coords.max() - x_coords.min()
y_range = y_coords.max() - y_coords.min()

crop_coord = (
    x_coords.min() + 0.2 * x_range,
    x_coords.max(),
    y_coords.min() + 0.4 * y_range,
    y_coords.max()
)

sc.set_figure_params(figsize=(10, 10))

for pathway in pathways:
    pathway_name = pathway.replace('/', '_').replace('\\', '_').replace(':', '_').replace(' ', '_')
    
    # Generate spot vectors without H&E background
    sc.pl.spatial(
        adata,
        img_key=None,  # Remove H&E background
        color=pathway,
        vmax="p99",
        vmin="p5",
        size=0.8,
        color_map="RdBu",
        show=False,
        title=f'Reactome 2024: {pathway}',
        crop_coord=crop_coord,
        colorbar_loc="bottom"
    )
    plt.savefig(f'spot_plots/REACTOME_{pathway_name}.pdf', dpi=300, bbox_inches='tight')
    plt.close()
    
print(f"Generated {len(pathways)} Reactome pathway plots")

# Hallmark Pathways

In [6]:
import requests

url = "https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=MSigDB_Hallmark_2020"
outfile = "MSigDB_Hallmark_2020.gmt"

response = requests.get(url)

with open(outfile, "wb") as f:
    f.write(response.content)

print(f"Downloaded to {outfile}")

Downloaded to MSigDB_Hallmark_2020.gmt


In [7]:
vis_data = "/home/workspace/EXP-00971/TIS05393-001-010-EXP-00971-SB005/outs"

adata = sc.read_h5ad('filtered_adata.h5ad')

def gmt_to_decoupler(pth) -> pd.DataFrame:
    """Parse a gmt file to a decoupler pathway dataframe."""
    from itertools import chain, repeat
    from pathlib import Path

    pathways = {}

    with Path(pth).open("r") as f:
        for line in f:
            name, _, *genes = line.strip().split("\t")
            pathways[name] = genes

    return pd.DataFrame.from_records(
        chain.from_iterable(zip(repeat(k), v) for k, v in pathways.items()),
        columns=["geneset", "genesymbol"],
    )

hallmark = gmt_to_decoupler("MSigDB_Hallmark_2020.gmt")

hallmark_pathways = hallmark['geneset'].unique().tolist()

In [8]:
all_pathways = sorted(hallmark_pathways)

search_mapping = {
    "TNF-ALPHA SIGNALING VIA NF-KB": ["TNF", "NF-kappa B", "NF-κB", "tumor necrosis factor", "inflammatory", "apoptosis"],
    "IL-2/STAT5 SIGNALING": ["IL-2", "interleukin-2", "STAT5", "JAK-STAT", "cytokine", "T cell", "hematopoietic"],
    "MYC TARGETS V1": ["MYC", "cell cycle", "transcription", "E2F", "proliferation", "growth"],
    "IL-6/JAK/STAT3 SIGNALING": ["IL-6", "interleukin-6", "JAK-STAT", "STAT3", "cytokine", "inflammatory", "immune"],
    "MET Activates PTK2 Signaling": ["MET", "receptor tyrosine kinase", "RTK", "focal adhesion", "PI3K-Akt", "MAPK", "migration", "invasion"]
}

print("Searching Hallamrk 2020 pathways:")
print("="*80)

for category, keywords in search_mapping.items():
    print(f"\n{category}:")
    matches = []
    for pathway in all_pathways:
        if any(keyword.lower() in pathway.lower() for keyword in keywords):
            matches.append(pathway)
    
    if matches:
        for match in matches:
            print(f"  ✓ {match}")
    else:
        print(f"  ✗ No matches found")

print(f"\n{'='*80}")
print(f"Total pathways searched: {len(all_pathways)}")

Searching Hallamrk 2020 pathways:

TNF-ALPHA SIGNALING VIA NF-KB:
  ✓ Apoptosis
  ✓ Inflammatory Response
  ✓ TNF-alpha Signaling via NF-kB

IL-2/STAT5 SIGNALING:
  ✓ IL-2/STAT5 Signaling

MYC TARGETS V1:
  ✓ E2F Targets
  ✓ Myc Targets V1
  ✓ Myc Targets V2

IL-6/JAK/STAT3 SIGNALING:
  ✓ IL-6/JAK/STAT3 Signaling
  ✓ Inflammatory Response

MET Activates PTK2 Signaling:
  ✓ Bile Acid Metabolism
  ✓ Fatty Acid Metabolism
  ✓ Xenobiotic Metabolism
  ✓ heme Metabolism

Total pathways searched: 50


In [9]:
# Filter duplicates
hallmark = hallmark[~hallmark.duplicated(("geneset", "genesymbol"))]

# Filtering genesets - FIX: Add observed=True
geneset_size = hallmark.groupby("geneset", observed=True).size()
gsea_genesets = geneset_size.index[(geneset_size > 15) & (geneset_size < 500)]

decoupler.run_aucell(
    adata,
    hallmark,
    source="geneset",
    target="genesymbol",
    use_raw=False,
)

pathways = [
    "Apoptosis",
    "Inflammatory Response",
    "TNF-alpha Signaling via NF-kB",
    "IL-2/STAT5 Signaling",
    "E2F Targets",
    "Myc Targets V1",
    "Myc Targets V2",
    "IL-6/JAK/STAT3 Signaling",
    "Bile Acid Metabolism",
    "Fatty Acid Metabolism",
    "Xenobiotic Metabolism",
    "heme Metabolism"
]

adata.obs[pathways] = adata.obsm["aucell_estimate"][pathways]

In [ ]:
spatial_coords = adata.obsm['spatial']

x_coords = spatial_coords[:, 0]
y_coords = spatial_coords[:, 1]

x_mid = (x_coords.min() + x_coords.max()) / 2
y_mid = (y_coords.min() + y_coords.max()) / 2

crop_coord = (x_mid, x_coords.max(), y_mid, y_coords.max())

x_range = x_coords.max() - x_coords.min()
y_range = y_coords.max() - y_coords.min()

crop_coord = (
    x_coords.min() + 0.2 * x_range,
    x_coords.max(),
    y_coords.min() + 0.4 * y_range,
    y_coords.max()
)

sc.set_figure_params(figsize=(10, 10))

for pathway in pathways:
    pathway_name = pathway.replace('/', '_').replace('\\', '_').replace(':', '_').replace(' ', '_')
    
    # Generate spot vectors without H&E background
    sc.pl.spatial(
        adata,
        img_key=None,  # Remove H&E background
        color=pathway,
        vmax="p99",
        vmin="p5",
        size=0.8,
        color_map="RdBu",
        show=False,
        title=f'Hallmark 2020: {pathway}',
        crop_coord=crop_coord,
        colorbar_loc="bottom"
    )
    plt.savefig(f'spot_plots/HALLMARK_{pathway_name}.pdf', dpi=300, bbox_inches='tight')
    plt.close()
    
print(f"Generated {len(pathways)} Hallmark pathway plots")

# KEGG Pathways

In [11]:
import requests

url = "https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=KEGG_2021_Human"
outfile = "KEGG_2021_Human.gmt"

response = requests.get(url)

with open(outfile, "wb") as f:
    f.write(response.content)

print(f"Downloaded to {outfile}")

Downloaded to KEGG_2021_Human.gmt


In [12]:
vis_data = "/home/workspace/EXP-00971/TIS05393-001-010-EXP-00971-SB005/outs"

adata = sc.read_h5ad('filtered_adata.h5ad')

def gmt_to_decoupler(pth) -> pd.DataFrame:
    """Parse a gmt file to a decoupler pathway dataframe."""
    from itertools import chain, repeat
    from pathlib import Path

    pathways = {}

    with Path(pth).open("r") as f:
        for line in f:
            name, _, *genes = line.strip().split("\t")
            pathways[name] = genes

    return pd.DataFrame.from_records(
        chain.from_iterable(zip(repeat(k), v) for k, v in pathways.items()),
        columns=["geneset", "genesymbol"],
    )

kegg = gmt_to_decoupler("KEGG_2021_Human.gmt")

kegg_pathways = kegg['geneset'].unique().tolist()

In [13]:
# Get unique pathway names
all_pathways = sorted(kegg_pathways)

# Expanded search terms for hypoxia-related pathways
search_mapping = {
    "Hypoxia-related": ["hypoxia", "HIF", "oxygen", "anoxia", "VEGF", "glycolysis"],
    "Oxidative phosphorylation": ["oxidative", "phosphorylation", "respiratory", "electron transport"]
}

print("Searching KEGG 2021 pathways:")
print("="*80)

for category, keywords in search_mapping.items():
    print(f"\n{category}:")
    matches = []
    for pathway in all_pathways:
        # Case-insensitive search
        if any(keyword.lower() in pathway.lower() for keyword in keywords):
            matches.append(pathway)
    
    if matches:
        for match in matches:
            print(f"  ✓ {match}")
    else:
        print(f"  ✗ No matches found")

print(f"\n{'='*80}")
print(f"Total pathways searched: {len(all_pathways)}")

Searching KEGG 2021 pathways:

Hypoxia-related:
  ✓ Glycolysis / Gluconeogenesis
  ✓ HIF-1 signaling pathway
  ✓ VEGF signaling pathway

Oxidative phosphorylation:
  ✓ Oxidative phosphorylation

Total pathways searched: 320


In [14]:
# Filter duplicates
kegg = kegg[~kegg.duplicated(("geneset", "genesymbol"))]

# Filtering genesets - FIX: Add observed=True
geneset_size = kegg.groupby("geneset", observed=True).size()
gsea_genesets = geneset_size.index[(geneset_size > 15) & (geneset_size < 500)]

decoupler.run_aucell(
    adata,
    kegg,
    source="geneset",
    target="genesymbol",
    use_raw=False,
)

pathways = [
    "Glycolysis / Gluconeogenesis",
    "Oxidative phosphorylation",
    "HIF-1 signaling pathway",
    "VEGF signaling pathway"
]

adata.obs[pathways] = adata.obsm["aucell_estimate"][pathways]

In [ ]:
spatial_coords = adata.obsm['spatial']

x_coords = spatial_coords[:, 0]
y_coords = spatial_coords[:, 1]

x_mid = (x_coords.min() + x_coords.max()) / 2
y_mid = (y_coords.min() + y_coords.max()) / 2

crop_coord = (x_mid, x_coords.max(), y_mid, y_coords.max())

x_range = x_coords.max() - x_coords.min()
y_range = y_coords.max() - y_coords.min()

crop_coord = (
    x_coords.min() + 0.2 * x_range,
    x_coords.max(),
    y_coords.min() + 0.4 * y_range,
    y_coords.max()
)

sc.set_figure_params(figsize=(10, 10))

for pathway in pathways:
    pathway_name = pathway.replace('/', '_').replace('\\', '_').replace(':', '_').replace(' ', '_')
    
    # Generate spot vectors without H&E background
    sc.pl.spatial(
        adata,
        img_key=None,  # Remove H&E background
        color=pathway,
        vmax="p99",
        vmin="p5",
        size=0.8,
        color_map="RdBu",
        show=False,
        title=f'KEGG 2021: {pathway}',
        crop_coord=crop_coord,
        colorbar_loc="bottom"
    )
    plt.savefig(f'spot_plots/KEGG_{pathway_name}.pdf', dpi=300, bbox_inches='tight')
    plt.close()
    
print(f"Generated {len(pathways)} KEGG pathway plots")

# Encode TF ChIP-seq 2015 Pathways

In [16]:
import requests

url = "https://maayanlab.cloud/Enrichr/geneSetLibrary?mode=text&libraryName=ENCODE_TF_ChIP-seq_2015"
r = requests.get(url)

with open("ENCODE_TF_ChIP-seq_2015.gmt", "wb") as f:
    f.write(r.content)

In [17]:
vis_data = "/home/workspace/EXP-00971/TIS05393-001-010-EXP-00971-SB005/outs"

adata = sc.read_h5ad('filtered_adata.h5ad')

def gmt_to_decoupler(pth) -> pd.DataFrame:
    """Parse a gmt file to a decoupler pathway dataframe."""
    from itertools import chain, repeat
    from pathlib import Path

    pathways = {}

    with Path(pth).open("r") as f:
        for line in f:
            name, _, *genes = line.strip().split("\t")
            pathways[name] = genes

    return pd.DataFrame.from_records(
        chain.from_iterable(zip(repeat(k), v) for k, v in pathways.items()),
        columns=["geneset", "genesymbol"],
    )

encode = gmt_to_decoupler("ENCODE_TF_ChIP-seq_2015.gmt")

encode_pathways = encode['geneset'].unique().tolist()

In [18]:
# Get unique pathway names
all_pathways = sorted(encode_pathways)

# Search terms optimized for ENCODE transcription factor targets
search_mapping = {
    "E2F4 ENCODE": ["E2F4", "E2F"],
    "Fatty Acid Metabolism": ["PPARA", "PPARG", "SREBF", "fatty", "lipid", "metabolism"],
    "Oxidative phosphorylation": ["oxidative", "phosphorylation", "OXPHOS", "mitochondrial", "respiratory", "electron"],
    "Glycolysis": ["glycolysis", "glycolytic", "glucose", "HIF1A", "MYC"],
    "Interferon Alpha Response": ["interferon", "IFNA", "IFN", "STAT1", "STAT2", "IRF"],
    "Hypoxia": ["hypoxia", "HIF1A", "HIF", "EPAS1", "oxygen"],
    "ISG15 Antiviral Mechanism": ["ISG15", "ISG", "antiviral", "interferon stimulated", "STAT1", "IRF"]
}

print("Searching ENCODE 2015 pathways:")
print("="*80)

for category, keywords in search_mapping.items():
    print(f"\n{category}:")
    matches = []
    for pathway in all_pathways:
        # Case-insensitive search
        if any(keyword.lower() in pathway.lower() for keyword in keywords):
            matches.append(pathway)
    
    if matches:
        for match in matches:
            print(f"  ✓ {match}")
    else:
        print(f"  ✗ No matches found")

print(f"\n{'='*80}")
print(f"Total pathways searched: {len(all_pathways)}")

Searching ENCODE 2015 pathways:

E2F4 ENCODE:
  ✓ E2F1 HeLa-S3 hg19
  ✓ E2F4 CH12.LX mm9
  ✓ E2F4 GM12878 hg19
  ✓ E2F4 HeLa-S3 hg19
  ✓ E2F4 K562 hg19
  ✓ E2F4 MCF 10A hg19
  ✓ E2F4 MEL cell line mm9
  ✓ E2F4 myocyte mm9
  ✓ E2F6 A549 hg19
  ✓ E2F6 H1-hESC hg19
  ✓ E2F6 HeLa-S3 hg19
  ✓ E2F6 K562 hg19
  ✓ HA-E2F1 HeLa-S3 hg19
  ✓ HA-E2F1 MCF-7 hg19

Fatty Acid Metabolism:
  ✓ SREBF1 GM12878 hg19
  ✓ SREBF1 HepG2 hg19
  ✓ SREBF2 GM12878 hg19

Oxidative phosphorylation:
  ✗ No matches found

Glycolysis:
  ✓ MYC A549 hg19
  ✓ MYC CH12.LX mm9
  ✓ MYC GM12878 hg19
  ✓ MYC H1-hESC hg19
  ✓ MYC HeLa-S3 hg19
  ✓ MYC HepG2 hg19
  ✓ MYC K562 hg19
  ✓ MYC MCF 10A hg19
  ✓ MYC MCF-7 hg19
  ✓ MYC MEL cell line mm9
  ✓ MYC NB4 hg19
  ✓ MYC endothelial cell of umbilical vein hg19

Interferon Alpha Response:
  ✓ IRF1 K562 hg19
  ✓ IRF3 GM12878 hg19
  ✓ IRF3 HeLa-S3 hg19
  ✓ IRF3 HepG2 hg19
  ✓ IRF4 GM12878 hg19
  ✓ STAT1 GM12878 hg19
  ✓ STAT1 HeLa-S3 hg19
  ✓ STAT1 K562 hg19
  ✓ STAT2 K562 hg19

Hyp

In [19]:
encode = encode[~encode.duplicated(("geneset", "genesymbol"))]

geneset_size = encode.groupby("geneset", observed=True).size()
gsea_genesets = geneset_size.index[(geneset_size > 15) & (geneset_size < 500)]

decoupler.run_aucell(
    adata,
    encode,
    source="geneset",
    target="genesymbol",
    use_raw=False,
)

pathways = [
    "E2F1 HeLa-S3 hg19",
    "E2F4 CH12.LX mm9",
    "E2F4 GM12878 hg19",
    "E2F4 HeLa-S3 hg19",
    "E2F4 K562 hg19",
    "E2F4 MCF 10A hg19",
    "E2F4 MEL cell line mm9",
    "E2F4 myocyte mm9",
    "E2F6 A549 hg19",
    "E2F6 H1-hESC hg19",
    "E2F6 HeLa-S3 hg19",
    "E2F6 K562 hg19",
    "MYC A549 hg19",
    "MYC CH12.LX mm9",
    "MYC GM12878 hg19",
    "MYC H1-hESC hg19",
    "MYC HeLa-S3 hg19",
    "MYC HepG2 hg19",
    "MYC K562 hg19",
    "MYC MCF 10A hg19",
    "MYC MCF-7 hg19",
    "MYC MEL cell line mm9",
    "MYC NB4 hg19",
    "MYC endothelial cell of umbilical vein hg19",
    "IRF1 K562 hg19",
    "IRF3 GM12878 hg19",
    "IRF3 HeLa-S3 hg19",
    "IRF3 HepG2 hg19",
    "IRF4 GM12878 hg19",
    "STAT1 GM12878 hg19",
    "STAT1 HeLa-S3 hg19",
    "STAT1 K562 hg19",
    "STAT2 K562 hg19"
]

adata.obs[pathways] = adata.obsm["aucell_estimate"][pathways]

In [ ]:
spatial_coords = adata.obsm['spatial']

x_coords = spatial_coords[:, 0]
y_coords = spatial_coords[:, 1]

x_mid = (x_coords.min() + x_coords.max()) / 2
y_mid = (y_coords.min() + y_coords.max()) / 2

crop_coord = (x_mid, x_coords.max(), y_mid, y_coords.max())

x_range = x_coords.max() - x_coords.min()
y_range = y_coords.max() - y_coords.min()

crop_coord = (
    x_coords.min() + 0.2 * x_range,
    x_coords.max(),
    y_coords.min() + 0.4 * y_range,
    y_coords.max()
)

sc.set_figure_params(figsize=(10, 10))

for pathway in pathways:
    pathway_name = pathway.replace('/', '_').replace('\\', '_').replace(':', '_').replace(' ', '_')
    
    # Generate spot vectors without H&E background
    sc.pl.spatial(
        adata,
        img_key=None,  # Remove H&E background
        color=pathway,
        vmax="p99",
        vmin="p5",
        size=0.8,
        color_map="RdBu",
        show=False,
        title=f'ENCODE 2015: {pathway}',
        crop_coord=crop_coord,
        colorbar_loc="bottom"
    )
    plt.savefig(f'spot_plots/ENCODE_{pathway_name}.pdf', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Save spot vector data
    spot_data = {
        'x_coords': x_coords,
        'y_coords': y_coords,
        'pathway_values': adata.obs[pathway].values
    }
    spot_df = pd.DataFrame(spot_data)
    spot_df.to_csv(f'spot_vectors/ENCODE_{pathway_name}_spots.csv', index=False)
    
print(f"Generated {len(pathways)} ENCODE pathway plots and spot vectors")